In [8]:
from web3 import Web3
from solcx import compile_source, install_solc, set_solc_version

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
ADDRESS = "0xF0DB3b5e4CbF54920E7A454D3062f10bbD2EC1A3"
w3.eth.default_account = Web3.to_checksum_address(ADDRESS)

print("Using account:", w3.eth.default_account)

install_solc("0.8.19")
set_solc_version("0.8.19")


contract_source = """
pragma solidity ^0.8.19;
contract FeedbackIntegrity {
    struct FeedbackRecord {
        bytes32 feedbackHash;
        uint256 timestamp;
        address submittedBy;
    }

    FeedbackRecord[] public records;

    function storeFeedbackHash(bytes32 _hash) public {
        records.push(FeedbackRecord(_hash, block.timestamp, msg.sender));
    }

    function getCount() public view returns (uint256) {
        return records.length;
    }
}
"""

compiled = compile_source(contract_source)
contract_interface = compiled['<stdin>:FeedbackIntegrity']

Feedback = w3.eth.contract(
    abi=contract_interface['abi'],
    bytecode=contract_interface['bin']
)

tx_hash = Feedback.constructor().transact({
    "from": w3.eth.default_account,
    "gas": 3000000
})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi']
)

print("Contract deployed at:", tx_receipt.contractAddress)

Using account: 0xF0DB3b5e4CbF54920E7A454D3062f10bbD2EC1A3
Contract deployed at: 0xaF5b929Ba346e2d81E07B74879f54db313Ad2A25


In [43]:
import hashlib
import json

# Original feedback
original_feedback = {
    "question_1_rating": 5,
    "question_2_rating": 5,
    "question_3_rating": 5,
    "question_4_rating": 5,
    "question_5_rating": 5,
    "comment": "The instructor explains concepts clearly."
}

In [44]:
# Compute hash
original_hash = hashlib.sha256(json.dumps(original_feedback, sort_keys=True).encode()).hexdigest()

In [45]:
# Submit to blockchain
tx = contract.functions.storeFeedbackHash(bytes.fromhex(original_hash)).transact({
    "from": w3.eth.default_account,
    "gas": 300000
})
w3.eth.wait_for_transaction_receipt(tx)

AttributeDict({'transactionHash': HexBytes('0x51c65acbac936c82fec1e8f7ba72ed2da161ef14bf6fe5394da7cb3e9006a1b2'),
 'transactionIndex': 0,
 'blockNumber': 10,
 'blockHash': HexBytes('0x12fdd1a72fc4d669fbe5404f4d51c7761c85f7a2a0eb80c235430798d80aa2fc'),
 'from': '0xF0DB3b5e4CbF54920E7A454D3062f10bbD2EC1A3',
 'to': '0xaF5b929Ba346e2d81E07B74879f54db313Ad2A25',
 'cumulativeGasUsed': 93644,
 'gasUsed': 93644,
 'contractAddress': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 

In [46]:
# Get latest record from blockchain
latest_index = contract.functions.getCount().call() - 1
record = contract.functions.records(latest_index).call()
onchain_hash = record[0].hex()

# Verify integrity
if original_hash == onchain_hash:
    print("✅ Instructor feedback valid")
else:
    print("❌ Instructor feedback tampered")

✅ Instructor feedback valid


In [47]:
# Tamper feedback locally
tampered_feedback = original_feedback.copy()
tampered_feedback["question_1_rating"] = 1

# Recompute hash
tampered_hash = hashlib.sha256(json.dumps(tampered_feedback, sort_keys=True).encode()).hexdigest()

In [48]:
# Get latest record from blockchain
latest_index = contract.functions.getCount().call() - 1
record = contract.functions.records(latest_index).call()
onchain_hash = record[0].hex()

# Verify integrity
if tampered_hash == onchain_hash:
    print("✅ Instructor feedback valid")
else:
    print("❌ Instructor feedback tampered")

❌ Instructor feedback tampered


In [49]:
count = contract.functions.getCount().call() 
print("Total instructor feedback records on-chain:", count)

Total instructor feedback records on-chain: 5
